In [ ]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets
!pip install -U bertviz
!pip install -U umap-learn
!pip install -U sentencepiece
!pip install -U urllib3
!pip install -U py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 63.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Usi

In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''

Mounted at /content/drive


In [ ]:
#!pip install --upgrade datasets
from datasets import load_dataset

In [ ]:
dataset = load_dataset("cnn_dailymail" , version = '3.0.0', split='test',trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 11490
})

In [ ]:
!pip install bert-score
!pip install rouge-score
import torch
from datasets import load_dataset, load_metric
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from tqdm import tqdm
import nltk
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from bert_score import score as bert_score
nltk.download('punkt')
nltk.download('wordnet')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=058dbad49b5fbd37e0fb454920e420e6e78e27f55cd298af81a95febf7723a26
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
#initializing GPT-2 model and tokenizer
model_name = "GPT-2"
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token #as gpt does not have inbuilt padding
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Initialize metrics
rouge =load_metric('rouge', trust_remote_code=True)
meteor =load_metric('meteor', trust_remote_code=True)

# Parameters
batch_size= 16
results=[]

reference_summaries =[]
generated_summaries =[]

print(f"Processing {model_name}")

# Processing in batches
for i in tqdm(range(0, len(dataset), batch_size), desc=f"Processing Test Set with {model_name}"):
    batch = dataset.select(range(i, min(i + batch_size, len(dataset))))
    articles = batch['article']
    references = batch['highlights']

    #GPT-2 doesn't have padding tokens, handle each article individually
    generated_batch = []
    for article in articles:
        inputs = tokenizer.encode_plus(article, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
        input_ids = inputs['input_ids'].to(device)
        attention_mask = inputs['attention_mask'].to(device)
        #summary generated using GPT2
        summary_ids = model.generate(input_ids, max_new_tokens=60, num_beams=4, early_stopping=True,pad_token_id=tokenizer.eos_token_id, attention_mask=attention_mask)
        generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
        generated_batch.append(generated_summary)


    #storing the references and generated summaries
    reference_summaries.extend([nltk.word_tokenize(ref.lower()) for ref in references])
    generated_summaries.extend([nltk.word_tokenize(gen.lower()) for gen in generated_batch])

    rouge.add_batch(predictions=generated_batch, references=references)
    meteor.add_batch(predictions=generated_batch, references=references)

#BLEUscore
bleu_score = corpus_bleu([[ref] for ref in reference_summaries], generated_summaries, smoothing_function=SmoothingFunction().method7)
#ROUGEscore
rouge_result = rouge.compute()
#METEOR score
meteor_result = meteor.compute()
#cosinesimilarity
vectorizer = TfidfVectorizer().fit(references + generated_batch)
reference_vectors = vectorizer.transform(references)
generated_vectors = vectorizer.transform(generated_batch)
cosine_scores = cosine_similarity(reference_vectors, generated_vectors).diagonal().mean()
#BERTScore
P, R, F1 = bert_score(generated_batch, references, lang='en', verbose=True)
bert_score_f1 = F1.mean().item()

#results
results.append({
    "Model": model_name,
    "BLEU": bleu_score,
    "ROUGE-1": rouge_result['rouge1'].mid.fmeasure,
    "ROUGE-2": rouge_result['rouge2'].mid.fmeasure,
    "ROUGE-L": rouge_result['rougeL'].mid.fmeasure,
    "METEOR": meteor_result['meteor'],
    "Cosine Similarity": cosine_scores,
    "BERTScore": bert_score_f1
})
df = pd.DataFrame(results)
print(df)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.60 seconds, 3.34 sentences/sec
   Model      BLEU   ROUGE-1   ROUGE-2   ROUGE-L    METEOR  Cosine Similarity  \
0  GPT-2  0.126693  0.177573  0.096084  0.126382  0.324486            0.42111   

   BERTScore  
0   0.853581  


In [ ]:
import platform
import psutil
import torch
import os
import sys
import subprocess
# system information
system = platform.system()
release = platform.release()
version = platform.version()
architecture = platform.architecture()[0]
processor = platform.processor()
python_version = platform.python_version()
machine = platform.machine()
memory = psutil.virtual_memory()
cpu_count = psutil.cpu_count(logical=True)
kernel_version = os.uname()
# GPU information
if torch.cuda.is_available():
    cuda_version= torch.version.cuda
    gpu_name = torch.cuda.get_device_name(0)
    gpu_count= torch.cuda.device_count()
    gpu_capability= torch.cuda.get_device_capability(0)
    gpu_memory_total= torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)  # Convert to GB
else:
    cuda_version= "No CUDA available"
    gpu_name= "No GPU available"
    gpu_count= 0
    gpu_capability= "N/A"
    gpu_memory_total= "N/A"
try:
    nvcc_output = subprocess.check_output(["nvcc", "--version"]).decode("utf-8")
    nvcc_version = nvcc_output.split()[-2]
except:
    nvcc_version ="nvcc not installed"

installed_packages = subprocess.check_output([sys.executable, "-m", "pip", "list"]).decode("utf-8")

# prinitng the information
print(f"System: {system}")
print(f"Release: {release}")
print(f"Version: {version}")
print(f"Kernel Version: {kernel_version.sysname} {kernel_version.release}")
print(f"Architecture: {architecture}")
print(f"Processor: {processor}")
print(f"Python Version: {python_version}")
print(f"Machine: {machine}")
print(f"Total Memory: {memory.total / (1024 ** 3):.2f} GB")
print(f"Available Memory: {memory.available / (1024 ** 3):.2f} GB")
print(f"CPU Count: {cpu_count}")
print(f"GPU Name: {gpu_name}")
print(f"Number of GPUs: {gpu_count}")
print(f"CUDA Version (PyTorch): {cuda_version}")
print(f"CUDA Version (nvcc): {nvcc_version}")
print(f"GPU Capabilities: {gpu_capability}")
print(f"GPU Memory Total: {gpu_memory_total:.2f} GB")
print("\nSample of Installed Packages:\n", installed_packages[:500])  # Limit output to first 500 characters

System: Linux
Release: 6.1.85+
Version: #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
Kernel Version: Linux 6.1.85+
Architecture: 64bit
Processor: x86_64
Python Version: 3.10.12
Machine: x86_64
Total Memory: 50.99 GB
Available Memory: 47.26 GB
CPU Count: 8
GPU Name: Tesla T4
Number of GPUs: 1
CUDA Version (PyTorch): 12.1
CUDA Version (nvcc): Build
GPU Capabilities: (7, 5)
GPU Memory Total: 14.75 GB

Sample of Installed Packages:
 Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.33.0
aiohappyeyeballs                 2.3.5
aiohttp                          3.10.2
aiosignal                        1.3.1
alabaster                        0.7.16
albucore                         0.0.13
albumentations                   1.4.13
altair                           4.2.2
annotated-types                  0.7.0
anyio    
